In [10]:
import re
import urllib
import pandas as pd
import pyodbc
import sqlalchemy as sqla

import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor

def get_odbc_driver():
    # gets name of ODBC driver, with name "ODBC Driver <version> for SQL Server"
    drivers = [d for d in pyodbc.drivers() if 'ODBC Driver ' in d]
    
    if len(drivers) == 0:
        errmsg = f"ERROR. No usable ODBC Driver found for SQL Server." \
        f"drivers found include {drivers}. Check ODBC Administrator program" \
        "for more information."
        
        raise Exception (errmsg)
    else:
        d_versions = [re.findall('\d+', dv)[0] for dv in drivers] # [re.findall('\d+', dv)[0] for dv in drivers]
        latest_version = max([int(v) for v in d_versions])
        driver = f"ODBC Driver {latest_version} for SQL Server"
    
        return driver

driver = get_odbc_driver()

print("modules loaded.")

modules loaded.


In [59]:


test_tbl = "TEST_DC_ilut_combined2020_63_DPS"

ilut_cols = ['PARCELID', 'GISAc', 'JURIS', 'County', 'EJ_AREA', 'DU_TOT',
             'POP_TOT', 'HH_hh', 'ENR_K12', 'EMPTOT', 'EMPFOOD', 'EMPRET', 'EMPSVC', 'EMPIND', 
            'PT_TOT_RES', 'SOV_TOT_RES', 'HOV_TOT_RES', 'TRN_TOT_RES', 'BIK_TOT_RES', 'WLK_TOT_RES', 
             'VMT_TOT_RES']
eto_cols = ['PARCELID', 'LU']

query_str = """SELECT TOP 100 ilut.PARCELID, XCOORD, YCOORD, GISAc, ilut.JURIS, eto.LU
    FROM TEST_DC_ilut_combined2020_63_DPS ilut
        JOIN raw_eto2020_latest eto
            ON ilut.PARCELID = eto.PARCELID"""

output_tbl = r'I:\Projects\Darren\PPA3_GIS\PPA3_GIS.gdb\TEST_SQL_EXPORT'

#-----------------


servername = 'SQL-SVR'
dbname = 'MTP2024'
trustedconn = 'yes'

conn_str = f"DRIVER={driver};" \
        f"SERVER={servername};" \
        f"DATABASE={dbname};" \
        f"Trusted_Connection={trustedconn}"

conn_str = urllib.parse.quote_plus(conn_str)
engine = sqla.create_engine(f"mssql+pyodbc:///?odbc_connect={conn_str}")

df_itr = pd.read_sql_query(sql=query_str, con=engine, chunksize=10)

print("df chunk iterator created.")


df chunk iterator created.


In [60]:
for i, chunk in enumerate(df_itr):
    chunk_s = pd.DataFrame.spatial.from_xy(chunk, x_column='XCOORD', y_column='YCOORD', sr=2226)
    if i == 0:
        print(f"creating feature class {output_tbl}...")
        chunk_s.spatial.to_featureclass(output_tbl, sanitize_columns=False)
        out_tbl_fnames = [f.name for f in arcpy.ListFields(output_tbl)]
        print("loading rows...")
    else:
        drecs = chunk.to_dict(orient='records')
        fields_to_use = [f for f in out_tbl_fnames if f in chunk.columns]
        fields_to_use.append('SHAPE@XY')
        with arcpy.da.InsertCursor(output_tbl, field_names=fields_to_use) as inscur:
            for rec in drecs:
                coords = (float(rec['XCOORD']), float(rec['YCOORD']))
                row = [rec[fname] for fname in out_tbl_fnames if fname in fields_to_use] # put into correct output order
                row.append(coords)
                # import pdb; pdb.set_trace()
                inscur.insertRow(row)
                

print("completed.")

creating feature class I:\Projects\Darren\PPA3_GIS\PPA3_GIS.gdb\TEST_SQL_EXPORT...
loading rows...
completed.
